Perguntas:
- Quantos gêneros o filme "The Matrix" tem? Quais são os gêneros?
- Quantos gêneros cada filme tem, em média?
- Quais são os cinco filmes com maior número de gêneros?
- Quantos usuários deram nota para o filme "The Matrix"?
- Quantos usuários, em média, deram nota para cada filme?
- Quais são os cinco filmes para os quais mais usuários deram nota?
- Qual o rating médio do filme "The Matrix"?
- Cada filme tem um rating médio. Qual a média desses ratings, por gênero?
- Para cada gênero de filme, quantos usuários deram cada nota possível?

Dicas:
1. Só é necessário as tabelas movies e ratings para este projeto.
2. Pesquise a função "explode" no pandas. Ela pode ser útil.

## Importação das Bibliotecas

In [2]:
import pandas as pd
import numpy as np

## Extração dos Dados

In [3]:
def read_data_csv(filename=str):
    path_dir = 'dados/raw/movie_lens'
    df = pd.read_csv(f'{path_dir}/{filename}', decimal='.')
    return df

In [4]:
df_movies = read_data_csv('movies.csv')
df_ratings = read_data_csv('ratings.csv')

## Tratamento dos Dados

In [126]:
df_movies["ano"] = df_movies["title"].apply(lambda st: st[st.find("(")+1:st.find(")")])
df_movies["title"] = df_movies["title"].str.replace(r" \(.*\)","", regex=True)
df_movies['genres_count'] = df_movies['genres'].str.split('|').str.len()

In [127]:
# Concatenando a base de movies com o de ratings
id_title=df_movies[['movieId','title']]
movies_ratings = pd.merge(df_ratings, id_title, how = 'left', on = 'movieId')

In [128]:
# Criando a base de gêneros
movies_genres = df_movies.copy()
movies_genres['genres'] = movies_genres['genres'].str.split('|')
movies_genres = movies_genres.explode('genres')
movies_genres = movies_genres[['movieId','title', 'genres']]

## Exploração dos Dados

### Quantos gêneros o filme "The Matrix" tem? Quais são os gêneros?

In [129]:
df_aux = df_movies.copy()
df_aux["title"] = df_movies['title'].str.replace('[^A-Za-z0-9]+','', regex=True)
df_aux["title"] = df_aux["title"].str.lower()

In [130]:
matrix_id = df_aux[(df_aux["title"]== ("matrixthe"))]
matrix_id = matrix_id.iloc[0]['movieId']
df_aux[(df_aux['movieId'] == matrix_id)]['genres_count'].to_frame("Qtd_genres_matrix")

,Qtd_genres_matrix
1939,3


In [135]:
movies_genres[(movies_genres["movieId"] == matrix_id)]['genres'].to_frame("Gêneros")

,Gêneros
1939,Action
1939,Sci-Fi
1939,Thriller


### Quantos gêneros cada filme tem, em média?

In [136]:
genres_mean = df_movies["genres_count"].mean()
genres_mean

2.2668856497639087

### Quais são os cinco filmes com maior número de gêneros?

In [137]:
genres_highest = df_movies.nlargest(5, "genres_count")
genres_highest = genres_highest["title"].to_frame("Movies")
genres_highest

,Movies
7441,Rubber
5556,Patlabor: The Movie
1390,Mulan
2250,Who Framed Roger Rabbit?
3460,Osmosis Jones


### Quantos usuários deram nota para o filme "The Matrix"?

In [138]:
ratings_matrix_count = movies_ratings[(movies_ratings["movieId"] == matrix_id)]
ratings_matrix_count.count()["rating"]

278

### Quantos usuários, em média, deram nota para cada filme?

In [139]:
users_aver_movie = movies_ratings.groupby("title")["userId"].count().to_frame("users_average")
users_aver_movie['users_average'].mean()

10.730658720868362

### Quais são os cinco filmes para os quais mais usuários deram nota?

In [140]:
movies_ratings.groupby("title")["title"].count().to_frame("total_reviews").nlargest(5, "total_reviews")

,total_reviews
title,
Forrest Gump,329
"Shawshank Redemption, The",317
Pulp Fiction,307
"Silence of the Lambs, The",279
"Matrix, The",278


### Qual o rating médio do filme "The Matrix"?

In [141]:
ratings_matrix_mean = movies_ratings[(movies_ratings["movieId"] == matrix_id)]

In [142]:
ratings_matrix_mean["rating"].mean()

4.192446043165468

### Cada filme tem um rating médio. Qual a média desses ratings, por gênero?

In [143]:
mean_by_genres = movies_ratings.groupby("movieId")["rating"].mean().to_frame("rating_mean")
mean_by_genres = pd.merge(movies_genres, mean_by_genres, how = 'left', on = 'movieId')
mean_by_genres.groupby("genres")["rating_mean"].mean().to_frame("mean_genres")

,mean_genres
genres,
(no genres listed),3.371148
Action,3.094498
Adventure,3.215230
Animation,3.497119
Children,3.107690
Comedy,3.181716
Crime,3.301844
Documentary,3.781682
Drama,3.420905


### Para cada gênero de filme, quantos usuários deram cada nota possível?

In [144]:
id_title=movies_genres[['movieId','genres']]
df_aux = pd.merge(movies_ratings, id_title, how = 'left', on = 'movieId')
df_aux = df_aux[['userId', 'rating', 'genres']]

In [145]:
df_aux["rating"]=df_aux["rating"].astype(int)
df_aux = df_aux.groupby(['genres', 'userId'])['rating'].unique().to_frame("numb_unique_rating")
df_aux = df_aux['numb_unique_rating'].str.len().to_frame("diff_ratings").reset_index()

In [146]:
mask = df_aux['diff_ratings'] == 6
df_aux = df_aux[mask]

In [147]:
# Contandos os valores únicos
df_aux.groupby(['genres'])['userId'].count()

genres
Action       68
Adventure    58
Animation     9
Children     13
Comedy       83
Crime        36
Drama        72
Fantasy      36
Horror       15
IMAX          6
Musical       9
Mystery       9
Romance      44
Sci-Fi       46
Thriller     49
War           4
Western       2
Name: userId, dtype: int64